In [1]:
import praw
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords

In [2]:
start = time.time()

In [3]:
#No longer works, blocked by cloudflare :(
#Stock Tickers and Names

URL = 'https://stockanalysis.com/stocks/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'lxml')
wsb_tickers = soup.find_all('li')

wsb_ticker_list = []
wsb_name_list = []

for i in wsb_tickers[12:-18]:
    #print(i.text.split(" - "))
    wsb_ticker_list.append(i.text.split(" - ")[0])
    wsb_name_list.append(i.text.split(" - ")[1])
    
len(wsb_ticker_list), len(wsb_name_list)

(0, 0)

In [4]:
nasdaq = pd.read_csv('nasdaq_screener_1615222693757.csv')
wsb_ticker_list = []
wsb_name_list = []
for i in range(len(nasdaq)):
    wsb_ticker_list.append(nasdaq.Symbol[i])
    wsb_name_list.append(" ".join(nasdaq.Name[i].split()[0:-2]))

In [5]:
wsb_name_list

['Agilent Technologies Inc.',
 'Alcoa Corporation',
 'ATA Creativity Global American',
 'Artius Acquisition Inc. Class A',
 'Artius Acquisition',
 'Artius Acquisition',
 'Arlington Asset Investment Corp Class',
 'Arlington Asset Investment',
 'Arlington Asset Investment Corp 8.250% Seies C Fixed-to-Floating Rate Cumulative Redeemable',
 'American Airlines Group Inc.',
 'Altisource Asset Management',
 'Atlantic American Corporation',
 'Aarons Holdings Company Inc.',
 'Applied Optoelectronics Inc.',
 'AAON Inc.',
 'Advance Auto Parts Inc Advance Auto Parts',
 'Apple Inc.',
 'American Assets Trust Inc.',
 'Almaden Minerals Ltd.',
 'Atlas Air Worldwide Holdings NEW',
 'AllianceBernstein Holding',
 'ABB Ltd',
 'AbbVie Inc.',
 'AmerisourceBergen Corporation',
 'Ameris Bancorp',
 'AbCellera Biologics Inc.',
 'Abcam plc American',
 'Abeona Therapeutics Inc.',
 'Ambev S.A. American Depositary Shares (Each representing 1',
 'Asbury Automotive Group Inc',
 'ABG Acquisition Corp. I Class A',
 'ARC

In [6]:
len(wsb_ticker_list), len(wsb_name_list)

(7258, 7258)

In [7]:
#Crypto Tickers and Names

URL = 'https://coinmarketcap.com/all/views/all/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'lxml')
ssb_names = soup.find_all('td', 
                        class_ = 'cmc-table__cell cmc-table__cell--sticky cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__name')
ssb_name_list = []

for i in ssb_names:
    ssb_name_list.append(i.text.split('\">"')[0])

ssb_tickers = soup.find_all('td', 
                        class_ = 'cmc-table__cell cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__symbol')
ssb_ticker_list = []

for i in ssb_tickers:
    ssb_ticker_list.append(i.text.split('\">"')[0])

len(ssb_ticker_list), len(ssb_name_list)

(200, 200)

In [8]:
reddit = praw.Reddit(client_id='HG7dA6CRLvCD_w',
                     client_secret='rH5FP42F__la6jpUdt01BQvZU48WiA', 
                     user_agent='WSB_Trends')

stopwords_list = stopwords.words('english')

for ind in range(len(stopwords_list)):
    stopwords_list[ind] = stopwords_list[ind].upper()

def comments_scraper(sub, comment_age, hot, case_sensitive = False):
    posts = []
    subreddit = reddit.subreddit(sub)
    for post in subreddit.hot(limit = hot):
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
    posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

    comment_count = 0
    comments = ""

    for post_id in posts.id:
        submission = reddit.submission(id=post_id)
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            '''Loops through all comments. comment.body is a string with each comment's contents.
            comment.created is the time the comment was created.'''
            comment_age = (comment.created - time.time())/3600
            if comment_age <=comment_age:
                comments += comment.body + " "
                comment_count+=1
    if case_sensitive == False:
        comments = comments.upper()

    for character in'$ -.,\n!<>':
        comments = comments.replace(character, " ")

    comments = comments.split()
    comments_counter = Counter(comments)
    return(comments_counter)

In [9]:
wsb_comments

NameError: name 'wsb_comments' is not defined

In [ ]:
def wsb_leaderboard(n, hours, hot):
    wsb_comments = comments_scraper('wallstreetbets', hours, hot, case_sensitive = True)

    frequencies = []
    for tick in wsb_ticker_list:
        if tick not in stopwords_list:
            frequencies.append(wsb_comments[tick])
        else:
            frequencies.append(0)

    wsb_tickers_and_counts = pd.DataFrame([wsb_name_list, wsb_ticker_list, frequencies]).T
    wsb_tickers_and_counts.columns = ['stock','ticker','frequency']
    wsb_tickers_and_counts = wsb_tickers_and_counts.sort_values(by = ['frequency'], ascending = False)[0:n]
    wsb_tickers_and_counts.index = range(1,n+1)
    return wsb_tickers_and_counts

In [ ]:
def ssb_leaderboard(n, hours, hot):
    ssb_comments = comments_scraper('satoshistreetbets', hours, hot, case_sensitive = True)

    frequencies = []

    for coin in range(len(ssb_name_list)):
        freq = 0
        #print(name_list[coin], ticker_list[coin])
        if ssb_name_list[coin] not in stopwords_list:
            freq+=ssb_comments[ssb_name_list[coin]]
        else:
            freq+=0
        if ssb_ticker_list[coin] not in stopwords_list:
            freq+=ssb_comments[ssb_ticker_list[coin]]
        else:
            freq+=0        

        frequencies.append(freq)

    ssb_tickers_and_counts = pd.DataFrame([ssb_name_list, ssb_ticker_list, frequencies]).T
    ssb_tickers_and_counts.columns = ['stock','ticker','frequency']
    ssb_tickers_and_counts = ssb_tickers_and_counts.sort_values(by = ['frequency'], ascending = False)[0:n]
    ssb_tickers_and_counts.index = range(1,n+1)
    return ssb_tickers_and_counts
    


In [ ]:
wsb = wsb_leaderboard(10,6,10)
ssb = ssb_leaderboard(10,6,10)

In [ ]:
wsb

In [ ]:
ssb

In [ ]:
end = time.time()
print("Done in", round((end - start), 2), "seconds.")

In [ ]:
#Test of whether more hours worth of comments is slowing down scrape - no
for X in [1,2,4,6]:
    scrape_start = time.time()
    wsb_comments = comments_scraper('wallstreetbets', X, 10, case_sensitive = True)
    scrape_end = time.time()
    scrape_time = scrape_end - scrape_start
    print(X, round(scrape_time,4), 'seconds.')

In [ ]:
#Test of whether more hot posts worth of comments is slowing down scrape - yes
for X in [5,10,20,40]:
    scrape_start = time.time()
    wsb_comments = comments_scraper('wallstreetbets', 6, X, case_sensitive = True)
    scrape_end = time.time()
    scrape_time = scrape_end - scrape_start
    print(X, round(scrape_time,4), 'seconds.')